In [1]:
import pandas as pd
import numpy as np
import glob, os
import time
import pickle
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import LSTM, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.optimizers import SGD, RMSprop

np.random.seed(0)

with open('preprocess.pickle', 'rb') as handle:
    [X,  Y] = pickle.load(handle)
Y.drop(['RecordID'],1, inplace=True)
Y = np.array(Y)
ynew = np.zeros((Y.shape[0],2))
ynew[:,1] = Y[:,0]
ynew[:,0] = 1-Y[:,0]
Y = ynew

Using TensorFlow backend.


In [6]:
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cross_validation import StratifiedKFold
import pandas as pd;
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import StratifiedKFold
import numpy

best_params = None
best_score = 1000
auc = None
model_final = None
def score(params):
    global best_score
    global best_params
    global auc
    global model_final
    print "Training with params : "
    print params
    lstm_size = params['lstm_size']
    dropout = params['dropout']
    dense1 = params['dense1']
    dense2 = params['dense2']
    #dense3 = params['dense3']
    seed = 7
    numpy.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []
    for train, test in kfold.split(X, Y[:,0]):
        model = Sequential()
        model.add(LSTM(lstm_size, return_sequences = True, input_dim = 37, input_length = 49))
        model.add(Dropout(dropout))
        #print model.output_shape
        #model.add(LSTM(14, return_sequences = True))
        #model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(BatchNormalization())
        #print model.output_shape
        model.add(Dense(dense1, init='glorot_uniform', activation='relu'))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
        model.add(Dense(dense2, init='glorot_uniform', activation='relu'))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
        model.add(Dense(2, init='glorot_uniform', activation='relu'))
        model.add(BatchNormalization())
        model.add(Activation('softmax'))
        rmsprop = RMSprop()
        
        model.compile(loss='binary_crossentropy', optimizer=rmsprop, metrics=['accuracy'])
        model.fit(X[train], Y[train], nb_epoch=10, batch_size=40, verbose = 0)
        test_predict = model.predict(X[test],  batch_size=40)
        from sklearn.metrics import roc_auc_score
        score = roc_auc_score(Y[test], test_predict)
        #print test_predict.shape
        #print(score)
        cvscores.append(score)
    score = 1.0 - numpy.mean(cvscores)
    #scores = cross_validation.cross_val_score(clf, X, y, scoring='roc_auc', cv=sss, n_jobs=-1, verbose=0)
    #score = 1.0-scores.mean()
    if best_score > score:
        best_score = score
        best_params = params
        auc = 1.0 - score
        model_final = model
    print "AUC : "+str(1.0 - score)+",  Score/Loss : "+str(score)
    print "END\n\n"
    return {'loss': score, 'status': STATUS_OK}

def optimize(trials):
    space = {
    'lstm_size': hp.choice('lstm_size', np.arange(15, 35, 5, dtype=int)),
    "dropout": hp.choice("dropout",[0.5, 0.3]),
    'dense1': hp.choice("dense1", [70, 80, 90, 100, 125]),
    'dense2' : hp.choice('dense2', np.arange(8, 20, 3, dtype=int)),
    #"dense3": hp.choice('dense3', np.arange(5, 8, 1, dtype=int))
    }
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=50)
    #print "optimize best", best
    return best



# #Trials object where the history of search will be stored
trials = Trials()
best = optimize(trials)
#print "best", best
print "best_score : ", best_score
print "best_params\n\n", best_params
print "\n\nauc : ", auc

Training with params : 
{'dropout': 0.3, 'lstm_size': 25, 'dense1': 80, 'dense2': 14}
AUC : 0.78191980297,  Score/Loss : 0.21808019703
END


Training with params : 
{'dropout': 0.3, 'lstm_size': 15, 'dense1': 80, 'dense2': 11}
AUC : 0.788673175605,  Score/Loss : 0.211326824395
END


Training with params : 
{'dropout': 0.3, 'lstm_size': 25, 'dense1': 70, 'dense2': 17}
AUC : 0.782326646765,  Score/Loss : 0.217673353235
END


Training with params : 
{'dropout': 0.3, 'lstm_size': 25, 'dense1': 90, 'dense2': 17}
AUC : 0.780532914725,  Score/Loss : 0.219467085275
END


Training with params : 
{'dropout': 0.5, 'lstm_size': 15, 'dense1': 125, 'dense2': 14}
AUC : 0.746282242763,  Score/Loss : 0.253717757237
END


best_score :  0.211326824395
best_params

{'dropout': 0.3, 'lstm_size': 15, 'dense1': 80, 'dense2': 11}


auc :  0.788673175605
